In [122]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sn

In [162]:
koopModel = pickle.load(open('combinatorial_promoters/combinatorial_promoters_deepDMD_model.pickle','rb'))

In [163]:
Kx = koopModel['Kx'].T
Ku = koopModel['Ku'].T

In [164]:
print(Kx.shape)
print(Ku.shape)

(41, 41)
(41, 7)


# Steady State programming

In [188]:
maxState = 1
P = 0.0*np.zeros(shape=(1,Kx.shape[0]))
P[0,maxState] = 1.0; 
P

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]])

## convex optimization solver

In [189]:
from cvxpy import Minimize
from cvxpy import Minimize, Problem, Variable,norm1,installed_solvers,lambda_max;
from cvxpy import norm as cvxpynorm;
import cvxpy


optimal_u = Variable(shape=(Ku.shape[1],1))

optimal_Px = P*(np.matmul(np.linalg.inv(np.eye(Kx.shape[0]) - Kx),Ku)*optimal_u);
objective = Minimize(-P *(np.matmul(np.linalg.inv(np.eye(Kx.shape[0]) - Kx),Ku)*optimal_u))
constraints =[optimal_u>=0 ,25>=optimal_u]
prob = Problem(objective, constraints)
result = prob.solve(verbose=True)#(solver=solver_instance);
print(prob.status);

optimal_u_val = optimal_u.value ;
optimal_u_val[np.abs(optimal_u_val)<1e-5] = 0;
print("Optimal input: " + repr(optimal_u_val))
print("Objective value: " +repr(objective.value))
print("Optimal Px value: " + repr(optimal_Px.value[0][0]))

-----------------------------------------------------------------
           OSQP v0.5.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2018
-----------------------------------------------------------------
problem:  variables n = 7, constraints m = 14
          nnz(P) + nnz(A) = 14
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on

iter   objective    pri res    dua res    rho        time
   1  -2.9159e+03   8.00e+00   1.04e+02   1.00e-01   5.41e-05s
 100  -3.2367e+03   1.08e-06   1.31e-05   1.94e-02   4.44e-05s
plsh  -3.2367e+03   1.00e-30   5.67e-14   --------   1.17e-04s

st

## constrained nonlinear optimization

In [190]:
testMat = np.dot(np.eye(Kx.shape[0]) - Kx, Ku)

In [191]:
np.dot(np.dot(P,testMat),x0)

array([2.60972869])

In [194]:
maxState

1

In [193]:
from scipy.optimize import minimize

uMin = 0.01
uMax = 1.0

def objective(x):
    return -np.dot(np.dot(P, np.dot(np.eye(Kx.shape[0]) - Kx, Ku)), x)

# initial guesses
n = Ku.shape[1]
x0 = np.random.uniform(uMax/2,uMax,n)

# show initial objective
print('Initial objective: ' + str(objective(x0)))

# optimize
b = (uMin,uMax)
bnds = (b,)*n
# con1 = {'type': 'ineq', 'fun': constraint1} 
# con2 = {'type': 'eq', 'fun': constraint2}
# cons = ([con1,con2])
solution = minimize(objective,x0,method='SLSQP',bounds=bnds)
x = solution.x

# show final objective
print('Final objective: ' + str(objective(x)))

print(solution)
print('Solution')
print('u = ' + str(x))

Initial objective: [-0.16659878]
Final objective: [-0.67794287]
     fun: -0.6779428692982848
     jac: array([-0.11355195,  0.1515477 ,  0.07920132, -0.23260432, -0.12513237,
        0.15765423, -0.21053827])
 message: 'Optimization terminated successfully.'
    nfev: 54
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([1.  , 0.01, 0.01, 1.  , 1.  , 0.01, 1.  ])
Solution
u = [1.   0.01 0.01 1.   1.   0.01 1.  ]


# Reachability problem 
